In [1]:
pip install termcolor

  Obtaining dependency information for termcolor from https://files.pythonhosted.org/packages/d9/5f/8c716e47b3a50cbd7c146f45881e11d9414def768b7cd9c5e6650ec2a80a/termcolor-2.4.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import copy
from termcolor import colored

class Piece:
    # класс 'Piece' представляет общую шахматную фигуру с атрибутами для ее цвета и символа.
    def __init__(self, color, symbol):
        # конструктор (__init__) инициализирует экземпляр класса, задавая цвет и символ фигуры.
        self.color = color  # цвет фигуры ('white' или 'black').
        self.symbol = symbol  # символ, представляющий фигуру на доске.

    def __str__(self):
        # метод __str__ переопределяется, чтобы возвращать символ фигуры вместо описания объекта класса.
        return self.symbol

class Pawn(Piece):
    # класс 'Pawn' представляет пешку, наследуя от класса 'Piece'.
    def __init__(self, color):
        # конструктор класса 'Pawn', вызывает конструктор родительского класса 'Piece', передавая ему цвет и символ ('P' для белых и 'p' для черных пешек).
        super().__init__(color, 'P' if color == 'white' else 'p')

    # метод для получения доступных ходов пешки на основе ее текущего положения на доске.
    def get_valid_moves(self, board, position):
        valid_moves = []  # список для хранения возможных ходов.
        x, y = position  # текущие координаты пешки на доске.

        # логика определения ходов для белых пешек.
        if self.color == 'white':
            # проверяем, свободна ли клетка перед пешкой для хода.
            if board[x - 1][y] == '.':
                valid_moves.append((x - 1, y))
            # если пешка находится на своей начальной позиции, проверяем возможность двойного хода.
            if x == 6 and board[x - 2][y] == '.':
                valid_moves.append((x - 2, y))
            # проверяем возможность "съесть" фигуру противника по диагонали.
            if y > 0 and board[x - 1][y - 1] != '.' and board[x - 1][y - 1].color != self.color:
                valid_moves.append((x - 1, y - 1))
            if y < 7 and board[x - 1][y + 1] != '.' and board[x - 1][y + 1].color != self.color:
                valid_moves.append((x - 1, y + 1))

        # логика определения ходов для черных пешек.
        else:
            # аналогичные проверки для черных пешек, но в противоположном направлении.
            if board[x + 1][y] == '.':
                valid_moves.append((x + 1, y))
            if x == 1 and board[x + 2][y] == '.':
                valid_moves.append((x + 2, y))
            if y > 0 and board[x + 1][y - 1] != '.' and board[x + 1][y - 1].color != self.color:
                valid_moves.append((x + 1, y - 1))
            if y < 7 and board[x + 1][y + 1] != '.' and board[x + 1][y + 1].color != self.color:
                valid_moves.append((x + 1, y + 1))

        return valid_moves  # возвращаем список возможных ходов.

class Rook(Piece):
    # класс 'Rook' представляет ладью и наследует от класса 'Piece'.
    def __init__(self, color):
        # конструктор класса 'Rook', вызывает конструктор базового класса 'Piece', задавая цвет и символ ('R' для белых и 'r' для черных ладей).
        super().__init__(color, 'R' if color == 'white' else 'r')
        # атрибут 'has_moved' указывает, был ли ход сделан ладьей, это необходимо для реализации правила рокировки.
        self.has_moved = False

    def get_valid_moves(self, board, position):
        # метод для получения возможных ходов ладьи.
        valid_moves = []  # список для хранения доступных ходов.
        x, y = position  # текущие координаты ладьи на доске.

        # перебираем ходы вверх от текущей позиции ладьи.
        for i in range(x - 1, -1, -1):
            if board[i][y] == '.':
                # если клетка пуста, добавляем в возможные ходы.
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                # если в клетке фигура противника, добавляем ход и прекращаем цикл.
                valid_moves.append((i, y))
                break
            else:
                # если встречаем свою фигуру, прекращаем цикл.
                break

        # перебираем ходы вниз от текущей позиции.
        for i in range(x + 1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break

        # перебираем ходы влево от текущей позиции.
        for j in range(y - 1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break

        # перебираем ходы вправо от текущей позиции.
        for j in range(y + 1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break

        return valid_moves  # возвращаем список возможных ходов.

class Knight(Piece):
    # класс 'Knight' представляет шахматного коня и наследует от класса 'Piece'.
    def __init__(self, color):
        # Конструктор класса. Вызывает конструктор базового класса, задавая цвет и символ ('N' для белых и 'n' для черных коней).
        super().__init__(color, 'N' if color == 'white' else 'n')

    def get_valid_moves(self, board, position):
        # метод для получения возможных ходов коня.
        valid_moves = []  # список для хранения доступных ходов.
        x, y = position  # текущие координаты коня на доске.

        # возможные ходы коня, описывающие движение буквой "Г".
        moves = [(x - 2, y - 1), (x - 2, y + 1), (x - 1, y - 2), (x - 1, y + 2),
                 (x + 1, y - 2), (x + 1, y + 2), (x + 2, y - 1), (x + 2, y + 1)]

        for move in moves:
            # проверяем, что ход находится в пределах доски и либо ведет на пустую клетку, либо на клетку, занятую фигурой противоположного цвета.
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and (
                    board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)

        return valid_moves  # возвращаем список возможных ходов.

class Bishop(Piece):
    # класс 'Bishop' представляет шахматного слона и наследует от класса 'Piece'.
    def __init__(self, color):
        # конструктор класса, вызывает конструктор базового класса, задавая цвет и символ ('B' для белых и 'b' для черных слонов).
        super().__init__(color, 'B' if color == 'white' else 'b')

    def get_valid_moves(self, board, position):
        # метод для получения возможных ходов слона.
        valid_moves = []  # список для хранения доступных ходов.
        x, y = position  # текущие координаты слона на доске.

        # используем функцию zip для одновременного перебора по двум диапазонам, что позволяет проверять диагональные ходы.

        # ходы по левым диагоналям.
        for i, j in zip(range(x - 1, -1, -1), range(y - 1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x + 1, 8), range(y - 1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break

        # ходы по правым диагоналям.
        for i, j in zip(range(x - 1, -1, -1), range(y + 1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x + 1, 8), range(y + 1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break

        return valid_moves  # возвращаем список возможных ходов.

class Queen(Piece):
    # класс 'Queen' представляет шахматного ферзя и наследует от класса 'Piece'.
    def __init__(self, color):
        # конструктор класса 'Queen'. Вызывает конструктор базового класса 'Piece', задавая цвет и символ ('Q' для белых и 'q' для черных ферзей).
        super().__init__(color, 'Q' if color == 'white' else 'q')

    def get_valid_moves(self, board, position):
        # метод для определения возможных ходов ферзя.
        valid_moves = []  # список для хранения доступных ходов.
        x, y = position  # текущие координаты ферзя на доске.

        # перебираем ходы вверх и вниз от текущей позиции ферзя.
        for i in range(x - 1, -1, -1):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        for i in range(x + 1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break

        # перебираем ходы влево и вправо от текущей позиции.
        for j in range(y - 1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        for j in range(y + 1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break

        # перебираем диагональные ходы, используя функцию zip для одновременного перебора по двум направлениям.

        # ходы по левым диагоналям.
        for i, j in zip(range(x - 1, -1, -1), range(y - 1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x + 1, 8), range(y - 1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break

        # ходы по правым диагоналям.
        for i, j in zip(range(x - 1, -1, -1), range(y + 1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x + 1, 8), range(y + 1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break

        return valid_moves  # Возвращаем список возможных ходов.

class King(Piece):
    # класс 'King' представляет шахматного короля и наследует от класса 'Piece'.
    def __init__(self, color):
        # конструктор класса. Вызывает конструктор базового класса 'Piece', задавая цвет и символ ('K' для белых и 'k' для черных королей).
        super().__init__(color, 'K' if color == 'white' else 'k')
        # атрибут 'has_moved' определяет, сделал ли король уже ход, что важно для рокировки.
        self.has_moved = False

    def get_valid_moves(self, board, position):
        # метод для определения возможных ходов короля.
        valid_moves = []  # список для хранения доступных ходов.
        x, y = position  # текущие координаты короля на доске.

        # определяем стандартные ходы короля на одну клетку во все стороны.
        moves = [(x - 1, y - 1), (x - 1, y), (x - 1, y + 1),
                 (x, y - 1), (x, y + 1),
                 (x + 1, y - 1), (x + 1, y), (x + 1, y + 1)]
        for move in moves:
            # проверяем, находится ли ход в пределах доски и ведет ли на пустую клетку или клетку, занятую фигурой противника.
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and (
                    board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)

        # проверяем возможность рокировки, если король не совершал ходов.
        if not self.has_moved:
            # рокировка справа. проверяем, находится ли на крайней правой клетке ладья, которая еще не совершала ходов, и свободны ли клетки между королем и ладьей.
            if board[x][7] != '.' and isinstance(board[x][7], Rook) and not board[x][7].has_moved:
                if all(board[x][col] == '.' for col in range(y + 1, 7)):
                    valid_moves.append((x, y + 2))  # добавляем ход для рокировки.

            # рокировка слева. аналогичная проверка для левой стороны.
            if board[x][0] != '.' and isinstance(board[x][0], Rook) and not board[x][0].has_moved:
                if all(board[x][col] == '.' for col in range(1, y)):
                    valid_moves.append((x, y - 2))  # добавляем ход для рокировки.

        return valid_moves  # возвращаем список возможных ходов.

class Board:
    # класс доски
    def __init__(self):
        # доска заполняется символом '.'
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.setup_board()
        self.history_move = []

    def setup_board(self):
        # расставляем белые фигуры
        self.board[7][0] = Rook('white')
        self.board[7][1] = Knight('white')
        self.board[7][2] = Bishop('white')
        self.board[7][3] = Queen('white')
        self.board[7][4] = King('white')
        self.board[7][5] = Bishop('white')
        self.board[7][6] = Knight('white')
        self.board[7][7] = Rook('white')
        for i in range(8):
            self.board[6][i] = Pawn('white')

        # расставляем черные фигуры
        self.board[0][0] = Rook('black')
        self.board[0][1] = Knight('black')
        self.board[0][2] = Bishop('black')
        self.board[0][3] = Queen('black')
        self.board[0][4] = King('black')
        self.board[0][5] = Bishop('black')
        self.board[0][6] = Knight('black')
        self.board[0][7] = Rook('black')
        for i in range(8):
            self.board[1][i] = Pawn('black')

    # функция для вывода доски
    def print_board(self):
        # названия столбцов
        print('  a b c d e f g h')
        # номера строк + фигуры
        for i in range(7, -1, -1):
            print(f'{i + 1} ', end='')
            for j in range(8):
                print(self.board[7 - i][j], end=' ')
            print()

    # вывод доски с выделением белых фигур под угрозой
    def print_board_white(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]

        print('  a b c d e f g h')
        for i in range(7, -1, -1):
            print(f'{i + 1} ', end='')
            for j in range(8):
                piece = self.board[7 - i][j]
                if piece != '.' and piece.color == 'black':  # если фигура черная, получаем для нее возможные ходы
                    moves = piece.get_valid_moves(self.board, (7 - i, j))
                    for move in moves:
                        # если возможно срубить белую фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]] != '.' and board[move[0]][move[1]] != piece.color:
                            board[move[0]][move[1]] = colored(board[move[0]][move[1]], 'red')
                print(board[7 - i][j], end=' ')
            print()

    # вывод доски с выделением черных фигур под угрозой
    def print_board_black(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]

        print('  a b c d e f g h')
        for i in range(7, -1, -1):
            print(f'{i + 1} ', end='')
            for j in range(8):
                piece = self.board[7 - i][j]
                if piece != '.' and piece.color == 'white':  # если фигура белая, получаем для нее возможные ходы
                    moves = piece.get_valid_moves(self.board, (7 - i, j))
                    for move in moves:
                        # если возможно срубить черную фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]] != '.' and board[move[0]][move[1]] != piece.color:
                            board[move[0]][move[1]] = colored(board[move[0]][move[1]], 'red')
                print(board[7 - i][j], end=' ')
            print()

    # функция движения фигуры
    def move_piece(self, start, end):
        # словарь для распознавания букв
        dictt = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
        # проверка, возможен ли ход
        if self.is_valid_move(start, end):
            # если ходит ладья или король, меняем значение has_moved на True
            if isinstance(self.board[8 - int(start[1])][dictt[start[0]]], Rook):
                Rook.has_moved = True
            if isinstance(self.board[8 - int(start[1])][dictt[start[0]]], King):
                King.has_moved = True
                # проверяем, делаем ли мы рокировку справа
                if end == 'g1' or end == 'g8':
                    # двигаем ладью
                    self.board[8 - int(start[1])][dictt['f']] = self.board[8 - int(start[1])][dictt['h']]
                    self.board[8 - int(start[1])][dictt['h']] = '.'

                # проверяем, делаем ли мы рокировку слева
                elif end == 'c1' or end == 'c8':
                    # двигаем ладью
                    self.board[8 - int(start[1])][dictt['d']] = self.board[8 - int(start[1])][dictt['a']]
                    self.board[8 - int(start[1])][dictt['a']] = '.'

            # двигаем фигуру
            self.board[8 - int(end[1])][dictt[end[0]]] = self.board[8 - int(start[1])][dictt[start[0]]]
            self.board[8 - int(start[1])][dictt[start[0]]] = '.'
            self.history_move.append([start, end])
            return True
        else:
            return False

    # функция, для проверки возможности хода
    def is_valid_move(self, start, end):
        # словарь для распознавания букв
        dictt = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
        x1, y1 = start
        x2, y2 = end
        # меняем значения на нужные
        # у отвечает за строку, х - за столбец
        x1, x2 = dictt[x1], dictt[x2]
        y1, y2 = 8 - int(y1), 8 - int(y2)
        start = y1, x1
        # проверка, в пределах доски ли ход
        if 0 <= x1 < 8 and 0 <= y1 < 8 and 0 <= x2 < 8 and 0 <= y2 < 8:
            piece = self.board[y1][x1]
            # проверка, есть ли фигура на стартовой позиции
            if piece != '.':
                # получаем возможные ходы для фигуры
                valid_moves = piece.get_valid_moves(self.board, start)
                # возращаем True, если конечная позиция в возможных ходах
                # False, если нельзя сходить в конечную позицию
                return (y2, x2) in valid_moves
        return False

    # функция для проверки шаха
    def is_check(self, color):
        # получаем координаты короля
        king_position = self.get_king_position(color)
        for i in range(8):
            for j in range(8):
                # проходимся по доске
                piece = self.board[i][j]
                # если находим фигуру другого цвета
                if piece != '.' and piece.color != color:
                    # проверяем есть ли координаты короля в возможных ходах для фигуры
                    valid_moves = piece.get_valid_moves(self.board, (i, j))
                    if king_position in valid_moves:
                        return True
        return False

    # функция для проверки мата
    def is_checkmate(self, color):
        # выполняется, только если король уже под шахом
        if not self.is_check(color):
            return False

        # проверяем возможные ходы для короля, чтобы избежать шаха
        king_position = self.get_king_position(color)
        valid_moves = self.board[king_position[0]][king_position[1]].get_valid_moves(self.board, king_position)
        piece = self.board[king_position[0]][king_position[1]]
        for move in valid_moves:
            # делаем временный ход, и проверем попадает ли он под шах
            self.board[move[0]][move[1]] = piece
            self.board[king_position[0]][king_position[1]] = '.'
            if not self.is_check(color):
                # возвращаем фигуры назад
                self.board[king_position[0]][king_position[1]] = piece
                self.board[move[0]][move[1]] = '.'
                return False
            # возвращаем фигуры назад
            self.board[king_position[0]][king_position[1]] = piece
            self.board[move[0]][move[1]] = '.'

        # проверяем возможные ходы для других фигур, которые спасаю короля от шаха
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color:
                    for move in piece.get_valid_moves(self.board, (i, j)):
                        # делаем временный ход
                        self.board[move[0]][move[1]] = piece
                        self.board[i][j] = '.'
                        if not self.is_check(color):
                            # возвращаем фигуры назад
                            self.board[i][j] = piece
                            self.board[move[0]][move[1]] = '.'
                            return False
                        # возвращаем фигуры назад
                        self.board[i][j] = piece
                        self.board[move[0]][move[1]] = '.'
        return True

    # функция для получения координатов короля
    def get_king_position(self, color):
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color and isinstance(piece, King):
                    return (i, j)
        return None

    def rollback(self, cnt):
        # удаляем последние cnt ходов из истории ходов
        self.history_move = self.history_move[:-cnt]
        print('История ходов с учётом отката:', str(self.history_move))
        # создаем новую доску
        board = Board()
        # перемещаем фигуры на новой доске в соответствии с обновленной историей ходов
        for start, end in self.history_move:
            print('start, end --->', start, end)
            board.move_piece(start, end)
        # выводим обновленную доску
        board.print_board()  
        return board


# функция для игры
def play_game():
    # создаем доску
    board = Board()
    board.print_board()
    while True:
        # получаем ход белых
        while True:
            start = input("Ход белых (например, e2): ")
            if 'back' in start:
                cnt = int(start.split(' ')[1])
                board = board.rollback(cnt if (len(board.history_move) - cnt) % 2 == 0 else cnt+1)
                continue

            end = input("Ход белых (например, e4): ")
            if 'back' in end:
                cnt = int(start.split(' ')[1])
                board = board.rollback(cnt if (len(board.history_move) - cnt) % 2 == 0 else cnt+1)
                continue

            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим белыми
        board.move_piece(start, end)
        board.print_board_black()

        # проверяем, находятся ли черные под матом
        if board.is_checkmate('black'):
            print("Белые выиграли")
            break

        # проверяем, находятся ли черные под шахом
        if board.is_check('black'):
            print("У черных шах")

        # получаем ход черных
        while True:
            start = input("Ход черных (например, e7): ")
            if 'back' in start:
                cnt = int(start.split(' ')[1])
                board = board.rollback(cnt if (len(board.history_move) - cnt) % 2 == 0 else cnt+1)
                continue

            end = input("Ход черных (например, e5): ")
            if 'back' in end:
                cnt = int(start.split(' ')[1])
                board = board.rollback(cnt if (len(board.history_move) - cnt) % 2 == 0 else cnt+1)
                continue

            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим черными
        board.move_piece(start, end)
        board.print_board_white()

        # проверяем, находятся ли белые под матом
        if board.is_checkmate('white'):
            print("Черные выиграли!")
            break

        # проверяем, находятся ли белые под шахом
        if board.is_check('white'):
            print("У белых шах!")

play_game()

  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . . . . . 
3 . . . . . . . . 
2 P P P P P P P P 
1 R N B Q K B N R 


Ход белых (например, e2):  e2
Ход белых (например, e4):  e4


  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . P . . . 
3 . . . . . . . . 
2 P P P P . P P P 
1 R N B Q K B N R 


Ход черных (например, e7):  e7
Ход черных (например, e5):  e5


  a b c d e f g h
8 r n b q k b n r 
7 p p p p . p p p 
6 . . . . . . . . 
5 . . . . p . . . 
4 . . . . P . . . 
3 . . . . . . . . 
2 P P P P . P P P 
1 R N B Q K B N R 


Ход белых (например, e2):  e2
Ход белых (например, e4):  e4


Неправильный ход. Попробуйте снова.


KeyboardInterrupt: Interrupted by user